## MBA em Ciência de Dados
# Redes Neurais e Arquiteturas Profundas

### <span style="color:darkred">Módulo 8 - Introdução ao Aprendizado por Reforço</span>

#### <span style="color:darkred">**Parte 2: Algoritmo de Aprendizado por Reforço "Value Learning"**</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

In [ ]:
import gym
import numpy as np
from IPython.display import clear_output

### Problema Taxi

Relembrando:
* Temos 4 localizações relevantes indicadas por 0:R, 1:G, 2:Y e 3:B.
* Azul é o passageiro e magenta o destino.
* O taxi é amarelo quando livre e verde quando ocupado
* Grid contém 25 posições para o táxi, 5 para o passageiro, e 4 destinos

**Ações**:<br>
0 : mover para norte<br>
1 : mover para sul<br>
2 : mover para leste<br>
3 : mover para oeste<br>
4 : pegar passageiro (pickup)<br>
5 : deixar passageiro (dropoff)

**Observação/estado**:<br>
Posições do taxi, passageiro e destino codificada numericamente

**Recompensas**:<br>
* -1 : por passo<br>
* 20 : deixar passageiro<br>
* -10: executar "pegar" ou "deixar" ilegalmente

**Término**:
* Passageiro é deixado no destino

In [ ]:
env = gym.make("Taxi-v3")

# tabela Q
q_table = np.zeros([env.observation_space.n, env.action_space.n])

q_table.shape

(500, 6)

In [ ]:
for i in range(301,305):
    print(np.round(q_table[i],4))

[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]


#### Algoritmo de Value learning

Tentaremos predizer o valor atual, ajustando com o melhor valor do estado subsequente.

Iremos considerar uma taxa de aprendizado $\alpha$ e um desconto para recompensas futuras $\gamma$

In [ ]:
# hiperparametros
alpha = 0.1 # taxa de aprendizado
gamma = 0.5 # desconto de recompensas futuras

# historico
episodios = []

# episodios
for t in range(1, 101):
    s = env.reset()

    epochs, recompensas = 0, 0
    fim = False

    # episodio atual
    while not fim:
        # explorar espaco de acao
        a = env.action_space.sample()

        # realizar acao
        s_n, r, fim, info = env.step(a)
        # estado subsequente s_n

        # salvo o valor atual para (s,a) - Q(s,a)
        valor_ant = q_table[s,a]

        # verifica proximo valor
        prox_max = np.max(q_table[s_n])

        # combina com desconto na recompensa futura
        novo_valor = (1-alpha)*valor_ant + alpha*(r+gamma*prox_max)
        q_table[s,a] = novo_valor

        # atualiza estado
        s = s_n
        epochs += 1

    if (t % 250 == 0):
        clear_output(wait=True)
        print("Episódio: ", t)

Ao consultar a tabela, teremos *valores* diferentes

In [ ]:
for i in range(301,305):
    print(np.round(q_table[i],4))

[-1.1404 -1.059  -0.7172 -0.9744 -8.1059 -7.8449]
[-0.6963 -0.7288 -0.5557 -0.371  -3.4485 -4.149 ]
[-0.8611 -0.6747 -0.5985 -0.8149 -3.4815 -6.2218]
[-0.4231 -0.5191 -0.195  -0.6046 -5.7218 -3.4532]


### Avaliando a performance do agente

Após obter a função Q, armazenada na tabela, agora podemos avaliar a performance do agente

In [ ]:
n_episodios_teste = 50
total_epochs = 0
total_recs = 0

for i in range(n_episodios_teste):
    s = env.reset()
    epochs, rec_total_i = 0,0
    fim = False
    while not fim:
        a = np.argmax(q_table[s])
        s, r, fim, info = env.step(a)
        epochs += 1
        rec_total_i += r

    total_epochs += epochs
    total_recs += rec_total_i

print("Média de recompensas totais: %.2f" % (total_recs/n_episodios_teste))
print("Média de passos por episódio: %.2f" % (total_epochs/n_episodios_teste))

Média de recompensas totais: 8.42
Média de passos por episódio: 12.58


#### Executando uma vez para visualizar

In [ ]:
from time import sleep

def animacao_episodio(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print("t: ", (i + 1))
        print("Estado: ", frame['state'])
        print("Ação: ", frame['action'])
        print("Recompensa: ", frame['reward'])
        sleep(.5)

In [ ]:
# inicializacao
env.reset()
frames = [] # animacao
rec_total = 0
epochs = 0

s = env.reset()
epochs, rec_total_i = 0,0
fim = False
while not fim:
    a = np.argmax(q_table[s])
    s, r, fim, info = env.step(a)
    epochs += 1
    rec_total += r

    frames.append({
        'frame': env.render(mode='ansi'),
        'state': s,
        'action': a,
        'reward': r
        }
    )

env.close()

animacao_episodio(frames)
print("\nRecompensa total: ", rec_total)
print("Passos até o estado terminal: ", epochs)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

t:  23
Estado:  93
Ação:  1
Recompensa:  -1


KeyboardInterrupt: ignored